Problem with cos terms
They cannot be introduced at once. 
Even cos(theta1) and cos (Phi1 - theta3) cannot be included at once

In [1]:
from nmon import *

 /Users/nikolaygusarov/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py: 34

In [14]:
N = 1
M = 3
EC_shunt = 0.18 * 1

# choose sparse: 100/100, 100/10, 100/5, 100/3
# choose dense: 100/2

# nmon = Nmon(N=N, M=M, EJN=100*EC_shunt, EJM=100*EC_shunt, EC_shunt=EC_shunt)
nmon = Nmon(N=N, M=M, EJN=100*EC_shunt, EJM=100*EC_shunt, EC_shunt=EC_shunt)

nmon.hamiltonian_calc(flux=0.0, ng=[0, 0, 0], num_levels=6, make_plot=False, just_H=False, cutoff=3)#compute_cutoff(nmon.EJN, nmon.EJM, nmon.EC_total, cutoff_space=[2, 8]))

# print(nmon.transition_freqs)
print(nmon.bound_state_energies)
print(nmon.sym_hamiltonian)

[-41.0720067  -33.66328669 -24.54783566 -15.82560565 -15.54000406
 -14.47617627]
(-18.0*cos(θ1) - 18.0*cos((2πΦ_{1}) - θ3) - 18.0*cos(θ1 + θ2) - 18.0*cos(θ2 - θ3)) + (16.743708*n1**2 - 33.025167*n1*n2 - 16.512583*n1*n3 + 33.487417*n1*n_g1 - 33.025167*n1*n_g2 - 16.512583*n1*n_g3 + 16.974833*n2**2 + 16.974833*n2*n3 - 33.025167*n2*n_g1 + 33.949666*n2*n_g2 + 16.974833*n2*n_g3 + 16.743708*n3**2 - 16.512583*n3*n_g1 + 16.974833*n3*n_g2 + 33.487417*n3*n_g3 + 16.743708*n_g1**2 - 33.025167*n_g1*n_g2 - 16.512583*n_g1*n_g3 + 16.974833*n_g2**2 + 16.974833*n_g2*n_g3 + 16.743708*n_g3**2)


In [77]:
import numpy as np

def construct_hamiltonian_3island(n_cut, Phi_1=0.0, n_g1=0.0, n_g2=0.0, n_g3=0.0):
    """
    Construct the 3-island Hamiltonian in the charge basis (n1, n2, n3),
    implementing the polynomial (charging) and 4 cos(...) (Josephson) terms
    exactly as specified:

      H = [ -18*cos(θ1)
            - 18*cos((2πΦ1) - θ3)
            - 18*cos(θ1 + θ2)
            - 18*cos(θ2 - θ3) ]
          + [ 16.743708 n1^2 - 33.025167 n1*n2 - 16.512583 n1*n3 + ... + 16.743708 n_g3^2 ]

    Parameters
    ----------
    n_cut : int
        Charge cutoff for n1, n2, n3 in [-n_cut..n_cut]
    Phi_1 : float
        External flux (dimensionless, in units of Φ0) for the cos((2πΦ1)-θ3) term
    n_g1 : float
        Offset charge for island 1
    n_g2 : float
        Offset charge for island 2
    n_g3 : float
        Offset charge for island 3

    Returns
    -------
    H : np.ndarray
        Real NxN Hamiltonian matrix in the basis |n1, n2, n3>, dimension = (2*n_cut+1)^3
    """

    # 1) Build the basis of all (n1, n2, n3) states
    basis = []
    for n1 in range(-n_cut, n_cut + 1):
        for n2 in range(-n_cut, n_cut + 1):
            for n3 in range(-n_cut, n_cut + 1):
                basis.append((n1, n2, n3))
    dim = len(basis)

    # 2) Create a dictionary that maps (n1, n2, n3) -> matrix index
    n123_to_index = { triple: idx for idx, triple in enumerate(basis) }

    # 3) Initialize Hamiltonian
    H = np.zeros((dim, dim), dtype=complex)

    # 4) CHARGING ENERGY (polynomial) - placed on the diagonal
    #    We'll explicitly write out each polynomial term. 
    #    That polynomial includes offset charges n_g1, n_g2, n_g3 as well.
    for idx, (n1, n2, n3) in enumerate(basis):
        # offset-shifted variables if needed, but the polynomial is already expanded
        # so we just plug n1, n2, n3, n_g1, n_g2, n_g3 directly:

        # Terms from your final polynomial:
        # (We *combine* them in a single sum)
        poly_val = (
            16.743708*(n1**2) 
            - 33.025167*(n1*n2)
            - 16.512583*(n1*n3)
            + 33.487417*(n1*n_g1)
            - 33.025167*(n1*n_g2)
            - 16.512583*(n1*n_g3)
            + 16.974833*(n2**2)
            + 16.974833*(n2*n3)
            - 33.025167*(n2*n_g1)
            + 33.949666*(n2*n_g2)
            + 16.974833*(n2*n_g3)
            + 16.743708*(n3**2)
            - 16.512583*(n3*n_g1)
            + 16.974833*(n3*n_g2)
            + 33.487417*(n3*n_g3)
            + 16.743708*(n_g1**2)
            - 33.025167*(n_g1*n_g2)
            - 16.512583*(n_g1*n_g3)
            + 16.974833*(n_g2**2)
            + 16.974833*(n_g2*n_g3)
            + 16.743708*(n_g3**2)
        )

        H[idx, idx] = poly_val

    # # 5) JOSEPHSON (cos) TERMS
    # #    a) -18.0 * cos(θ1)
    # #       cos(θ1) = 0.5*( e^{+iθ1} + e^{-iθ1} ) => n1->n1±1
    # #       We'll incorporate the factor -18 => half => -9 for each ±1 hop
    # for idx, (n1, n2, n3) in enumerate(basis):
    #     coeff = -18.0 / 2.0   # -9.0
    #     for dn in [+1, -1]:
    #         n1p = n1 + dn
    #         if -n_cut <= n1p <= n_cut:
    #             idxp = n123_to_index.get((n1p, n2, n3))
    #             H[idx, idxp] += coeff  # add -9.0

    #    b) -18.0 * cos((2πΦ_1) - θ3)
    #       => shift n3->n3±1 with a phase factor e^{± i * 2πΦ1}
    #          specifically, for n3->n3+1 we multiply by e^{-i phi}
    #          for n3->n3-1 we multiply by e^{+i phi}
    phi = 2.0 * np.pi * Phi_1
    exp_minus_phi = np.exp(-1j * phi)  # for n3-> n3+1
    exp_plus_phi  = np.exp(+1j * phi)  # for n3-> n3-1
    for idx, (n1, n2, n3) in enumerate(basis):
        coeff = -18.0 / 2.0  # -9.0
        # n3 -> n3+1
        n3p = n3 + 1
        if -n_cut <= n3p <= n_cut:
            idxp = n123_to_index.get((n1, n2, n3p))
            H[idx, idxp] += coeff * exp_minus_phi
        # n3 -> n3-1
        n3p = n3 - 1
        if -n_cut <= n3p <= n_cut:
            idxp = n123_to_index.get((n1, n2, n3p))
            H[idx, idxp] += coeff * exp_plus_phi

    # #    c) -18.0 * cos(θ1 + θ2)
    # #       => shift (n1->n1±1, n2->n2±1) simultaneously
    # #          Each shift gets half => -9.0
    # for idx, (n1, n2, n3) in enumerate(basis):
    #     coeff = -18.0 / 2.0  # -9.0
    #     for delta in [+1, -1]:
    #         n1p = n1 + delta
    #         n2p = n2 + delta
    #         if -n_cut <= n1p <= n_cut and -n_cut <= n2p <= n_cut:
    #             idxp = n123_to_index[(n1p, n2p, n3)]
    #             H[idx, idxp] += coeff

    # #    d) -18.0 * cos(θ2 - θ3)
    # #       => shift n2->n2±1, n3->n3∓1
    # #          again half => -9.0 for each pair of ±1
    # for idx, (n1, n2, n3) in enumerate(basis):
    #     coeff = -18.0 / 2.0  # -9.0
    #     for d2 in [+1, -1]:
    #         d3 = -d2  # so if d2=+1 => d3=-1
    #         n2p = n2 + d2
    #         n3p = n3 + d3
    #         if -n_cut <= n2p <= n_cut and -n_cut <= n3p <= n_cut:
    #             idxp = n123_to_index[(n1, n2p, n3p)]
    #             H[idx, idxp] += coeff

    # 6) Make the Hamiltonian Hermitian
    H = 0.5 * (H + H.conj().T)

    # 7) Return the real part if you prefer a real matrix
    return H.real


# ---------------------------------------------------------
# EXAMPLE USAGE
if __name__ == "__main__":
    n_cut = 7   # or 7, etc.
    Phi_1 = 0.0
    n_g1 = 0.0
    n_g2 = 0.0
    n_g3 = 0.0

    # Build Hamiltonian
    H_matrix = construct_hamiltonian_3island(n_cut, Phi_1, n_g1, n_g2, n_g3)
    print("H matrix shape:", H_matrix.shape)

    # Diagonalize
    e_vals, e_vecs = np.linalg.eigh(H_matrix)
    # Sort & print
    sorted_eigs = np.sort(e_vals)
    print("Eigenvalues:", sorted_eigs[:6])

    # Example ratio
    max_diag = np.max(np.diag(H_matrix))
    size = H_matrix.shape[0]
    off_diag_element = H_matrix[0, 1]
    ratio = max_diag / (size**2) / off_diag_element
    print("Computed ratio:", ratio)

H matrix shape: (3375, 3375)
Eigenvalues: [-7.11761012 -6.42664281 -6.42664281 -4.35374568 -4.35374568 -0.89893304]
Computed ratio: -5.5911152743789067e-05


In [23]:
# import numpy as np

# def construct_hamiltonian_newvars(n_cut, Phi_1=0.0):
#     r"""
#     Construct the 3-island Hamiltonian in the *new* variables (p1, p2, p3),
#     but enumerating states in the *charge basis* for n_{p1}, n_{p2}, n_{p3}.
    
#     The transformation of angles is:
#       p1 = θ1,
#       p3 = θ3,
#       p2 = θ1 + θ2  => θ2 = p2 - p1.
    
#     Then the *old* charges relate to the new ones by:
#       n1 = n_{p1} + n_{p2},
#       n2 = n_{p2},
#       n3 = n_{p3}.
    
#     Cosine terms become:
#       -18 cos(p1),
#       -18 cos((2πΦ_1) - p3),
#       -18 cos(p2),
#       -18 cos(p2 - p1 - p3).
    
#     The big polynomial in n1, n2, n3 is replaced by the same expression
#     but with n1 = n_{p1}+n_{p2}, etc. That yields a diagonal in the
#     (n_{p1}, n_{p2}, n_{p3}) basis.

#     Parameters
#     ----------
#     n_cut : int
#         The cutoff for each of the new charges n_{p1}, n_{p2}, n_{p3}
#         in [-n_cut, ..., +n_cut].
#     Phi_1 : float
#         Dimensionless external flux (in units of Φ0). 
#         Enters the term cos((2πΦ_1) - p3).

#     Returns
#     -------
#     H : np.ndarray
#         A real, Hermitian matrix of dimension (2*n_cut+1)^3. We can diagonalize
#         via np.linalg.eigh(H).
#     """

#     # ---- 1) Build the charge-basis list for (n_{p1}, n_{p2}, n_{p3}) ----
#     basis = []
#     for np1 in range(-n_cut, n_cut + 1):
#         for np2 in range(-n_cut, n_cut + 1):
#             for np3 in range(-n_cut, n_cut + 1):
#                 basis.append((np1, np2, np3))
#     dim = len(basis)  # (2*n_cut+1)^3

#     # A lookup dict from (np1, np2, np3) to row/col index
#     index_map = {(a, b, c): idx for idx, (a, b, c) in enumerate(basis)}

#     # ---- 2) Initialize Hamiltonian ----
#     H = np.zeros((dim, dim), dtype=complex)

#     # ---- 3) Charging-energy function ----
#     def charging_energy(np1, np2, np3):
#         """
#         Example polynomial:
#           16.743708*n1^2 
#         + -33.025167*n1*n2 
#         + -16.512583*n1*n3
#         + 16.974833*n2^2
#         + 16.974833*n2*n3
#         + 16.743708*n3^2

#         with n1 = np1+np2, n2 = np2, n3 = np3.
#         (Add offset-charge terms if needed.)
#         """
#         n1 = np1 + np2
#         n2 = np2
#         n3 = np3
#         val = (
#               16.743708*(n1**2)
#             - 33.025167*(n1*n2)
#             - 16.512583*(n1*n3)
#             + 16.974833*(n2**2)
#             + 16.974833*(n2*n3)
#             + 16.743708*(n3**2)
#         )
#         return val

#     # ---- 4) Fill diagonal from the charging polynomial ----
#     for i, (np1, np2, np3) in enumerate(basis):
#         H[i, i] = charging_energy(np1, np2, np3)

#     # ---- 5) Add the 4 Josephson cos(...) terms ----
#     # Each has factor -18 => half => -9 for the ±1 charge hops

#     # 5a) -18 cos(p1) => shift n_{p1} -> n_{p1} ± 1
#     coeff = -9.0  # half of -18
#     for i, (np1, np2, np3) in enumerate(basis):
#         for delta in [+1, -1]:
#             np1p = np1 + delta
#             if -n_cut <= np1p <= n_cut:
#                 j = index_map.get((np1p, np2, np3))
#                 H[i, j] += coeff

#     # 5b) -18 cos((2πΦ_1) - p3) => shift n_{p3} -> n_{p3} ± 1 w/ flux phase
#     phi = 2.0 * np.pi * Phi_1
#     for i, (np1, np2, np3) in enumerate(basis):
#         for delta in [+1, -1]:
#             np3p = np3 + delta
#             if -n_cut <= np3p <= n_cut:
#                 j = index_map.get((np1, np2, np3p))
#                 # p3 -> p3+1 => factor e^{-i phi}, p3-> p3-1 => e^{+i phi}
#                 if delta == +1:
#                     H[i, j] += coeff * np.exp(-1j * phi)
#                 else:
#                     H[i, j] += coeff * np.exp(+1j * phi)

#     # 5c) -18 cos(p2) => shift n_{p2} -> n_{p2} ± 1
#     for i, (np1, np2, np3) in enumerate(basis):
#         for delta in [+1, -1]:
#             np2p = np2 + delta
#             if -n_cut <= np2p <= n_cut:
#                 j = index_map.get((np1, np2p, np3))
#                 H[i, j] += coeff

#     # 5d) -18 cos(p2 - p1 - p3)
#     #     => cos(x) = 0.5*(e^{i x} + e^{-i x}),
#     #        x = (p2 - p1 - p3).
#     #     => "raising" shift might be: p2 -> p2+1, p1 -> p1-1, p3 -> p3-1, etc.
#     # We'll define the two ± combos explicitly:
#     for i, (np1, np2, np3) in enumerate(basis):
#         # e^{+i (p2 - p1 - p3)} => 
#         #   n_{p2} -> n_{p2}+1, n_{p1} -> n_{p1}-1, n_{p3} -> n_{p3}-1
#         # e^{-i (p2 - p1 - p3)} => the opposite shifts:
#         combos = [
#             (+1, -1, -1),  # (delta_np2, delta_np1, delta_np3) for the "positive" exponent
#             (-1, +1, +1),  # the "negative" exponent
#         ]
#         for d2, d1, d3 in combos:
#             np1p = np1 + d1
#             np2p = np2 + d2
#             np3p = np3 + d3
#             if (
#                 -n_cut <= np1p <= n_cut
#                 and -n_cut <= np2p <= n_cut
#                 and -n_cut <= np3p <= n_cut
#             ):
#                 j = index_map[(np1p, np2p, np3p)]
#                 H[i, j] += coeff

#     # ---- 6) Symmetrize to ensure Hermitian ----
#     H = 0.5 * (H + H.conj().T)

#     # ---- 7) Return real part ----
#     return H.real


# # ---------- EXAMPLE USAGE ----------
# if __name__ == "__main__":
#     n_cut = 3
#     flux = 0.0
#     H_mat = construct_hamiltonian_newvars(n_cut, Phi_1=flux)
#     print("Shape of Hamiltonian:", H_mat.shape)

#     # Diagonalize
#     eigvals, eigvecs = np.linalg.eigh(H_mat)
#     idx_sort = np.argsort(eigvals)
#     eigvals = eigvals[idx_sort]
#     print("Eigenvalues (lowest few):", eigvals[:10])

#### 1-2 Hamiltonian



In [80]:
def construct_subH_n1n2(
    n_cut, n3_val, n_g1, n_g2, n_g3,
    # same polynomial coefficients as above
    c_n1n1=16.743708,
    c_n1n2=-33.025167,
    c_n1n3=-16.512583,
    c_n2n2=16.974833,
    c_n2n3=16.974833,
    c_n3n3=16.743708,
    # cross terms with offsets
    c_n1g1=33.487417,
    c_n1g2=-33.025167,
    c_n1g3=-16.512583,
    c_n2g1=-33.025167,
    c_n2g2=33.949666,
    c_n2g3=16.974833,
    c_n3g1=-16.512583,
    c_n3g2=16.974833,
    c_n3g3=33.487417,
    # offset^2 terms
    c_g1g1=16.743708,
    c_g1g2=-33.025167,
    c_g1g3=-16.512583,
    c_g2g2=16.974833,
    c_g2g3=16.974833,
    c_g3g3=16.743708
):
    """
    Build a 2D polynomial Hamiltonian in (n1,n2) for a *fixed* integer n3 = n3_val.
    We'll treat terms involving n3 as constants or linear in (n1,n2).
    """
    basis_12 = []
    for i1 in range(-n_cut, n_cut+1):
        for i2 in range(-n_cut, n_cut+1):
            basis_12.append((i1, i2))
    dim_12 = len(basis_12)
    H_12 = np.zeros((dim_12, dim_12), dtype=float)
    idx_map_12 = {(a,b): idx for idx, (a,b) in enumerate(basis_12)}

    # Evaluate (n3 - n_g3) once
    d3 = (n3_val - n_g3)

    for idx_12, (n1, n2) in enumerate(basis_12):
        d1 = (n1 - n_g1)
        d2 = (n2 - n_g2)

        # Gather polynomial terms that involve n1,n2 plus constant shifts from n3_val
        # For instance, c_n1n3*(n1*n3) => c_n1n3*(n1*(n3_val)) ...
        val = (
            c_n1n1*(d1**2)
            + c_n2n2*(d2**2)
            + c_n1n2*(d1*d2)
            # cross with n3_val is not from here
            + c_n1n3*(n1*n3_val)
            + c_n2n3*(n2*n3_val)
            + c_n1g1*(n1*n_g1) + c_n1g2*(n1*n_g2) + c_n1g3*(n1*n_g3)
            + c_n2g1*(n2*n_g1) + c_n2g2*(n2*n_g2) + c_n2g3*(n2*n_g3)
            + c_n3g1*(n3_val*n_g1) + c_n3g2*(n3_val*n_g2) + c_n3g3*(n3_val*n_g3)
            + c_g1g1*(n_g1**2) + c_g1g2*(n_g1*n_g2) + c_g1g3*(n_g1*n_g3)
            + c_g2g2*(n_g2**2) + c_g2g3*(n_g2*n_g3) + c_g3g3*(n_g3**2)
        )

        # Place on diagonal
        H_12[idx_12, idx_12] = val

    
    # # 4) Off-diagonal from cos(theta1) ~ => shift n1 -> n1 ± 1
    # #    EJ1 is the full amplitude (e.g. -18 => half => -9 for each ±1 hop)
    # #    cos(...) = 0.5*(e^{i theta1}+ e^{-i theta1})
    # coeff_1 = -18/2.0  # e.g. -9.0 if EJ1=-18
    # for idx_12, (n1, n2) in enumerate(basis_12):
    #     for dn in [+1, -1]:
    #         n1p = n1 + dn
    #         if -n_cut <= n1p <= n_cut:
    #             idxp = idx_map_12.get((n1p, n2))
    #             if idxp is not None:
    #                 H_12[idx_12, idxp] += coeff_1

    # # ->>>>> this does not work that easy since we got dependencies on n3 value
    # # 5) Off-diagonal from cos(theta1+theta2) => shift (n1->n1±1, n2->n2±1) simultaneously
    # coeff_12 = -18/2.0
    # for idx_12, (n1, n2) in enumerate(basis_12):
    #     for d in [+1, -1]:
    #         n1p = n1 + d
    #         n2p = n2 + d
    #         if (-n_cut <= n1p <= n_cut) and (-n_cut <= n2p <= n_cut):
    #             idxp = idx_map_12.get((n1p, n2p))
    #             if idxp is not None:
    #                 H_12[idx_12, idxp] += coeff_12

    return H_12, basis_12


# ---------- EXAMPLE USAGE ----------
if __name__ == "__main__":
    n_cut = 7
    n3_val = 0
    n_g1 = 0
    n_g2 = 0
    n_g3 = 0

    H_12, basis_12 = construct_subH_n1n2(n_cut, n3_val, n_g1, n_g2, n_g3)
    print("Shape of H_12:", H_12.shape)

    # Diagonalize
    eigvals, eigvecs = np.linalg.eigh(H_12)
    idx_sort = np.argsort(eigvals)
    eigvals = eigvals[idx_sort]
    print("Eigenvalues (lowest few):", eigvals[:6])

Shape of H_12: (225, 225)
Eigenvalues (lowest few): [0.       0.693374 0.693374 2.773496 2.773496 6.240366]


In [79]:
def subH_3_only(
    n_cut,
    n_g3,
    c_n3n3=16.743708,
    EJ3=-18.0,
    Phi_1=0.0
):
    """
    Build a sub-Hamiltonian for island #3 alone:
      - polynomial in (n3 - n_g3),
      - cos((2π Φ1) - theta3).
    We exclude cross terms that couple n3 to n1 or n2.

    Returns: H3 (1D), dimension = (2*n_cut+1) x (2*n_cut+1).
    """
    basis_3 = [i3 for i3 in range(-n_cut, n_cut+1)]
    dim_3 = len(basis_3)
    H3 = np.zeros((dim_3, dim_3), dtype=complex)
    idx_map_3 = {val: idx for idx, val in enumerate(basis_3)}

    # 1) Diagonal polynomial
    for i3, n3 in enumerate(basis_3):
        d3 = (n3 - n_g3)
        # Just c_n3n3*(d3^2), ignoring cross n1n3, n2n3, etc.
        val = c_n3n3*(d3**2)
        H3[i3, i3] = val

    # 2) Off-diagonal from cos((2πΦ1)-theta3) => shift n3-> n3±1 w/ phase factors
    coeff_3 = EJ3/2.0  # half of -18 => -9
    phi = 2.0*np.pi*Phi_1

    for i3, n3 in enumerate(basis_3):
        for dn in [+1, -1]:
            n3p = n3 + dn
            if -n_cut <= n3p <= n_cut:
                j3 = idx_map_3.get(n3p)
                if j3 is not None:
                    # cos((2πΦ1)-theta3) => e^{± i(2πΦ1)} factor
                    # For n3->n3+1 => e^{-i phi}, n3->n3-1 => e^{+i phi}
                    if dn == +1:
                        H3[i3, j3] += coeff_3 * np.exp(-1j*phi)
                    else:
                        H3[i3, j3] += coeff_3 * np.exp(+1j*phi)

    # 3) Hermitize
    H3 = 0.5*(H3 + H3.conj().T)
    return H3, basis_3

# ---------- EXAMPLE USAGE ----------
if __name__ == "__main__":
    n_cut = 7
    n_g3 = 0
    H3, basis_3 = subH_3_only(n_cut, n_g3)
    print("Shape of H3:", H3.shape)

    # Diagonalize
    eigvals, eigvecs = np.linalg.eigh(H3)
    idx_sort = np.argsort(eigvals)
    eigvals = eigvals[idx_sort]
    print("Eigenvalues (lowest few):", eigvals[:6])

Shape of H3: (15, 15)
Eigenvalues (lowest few): [ -7.11761012  15.16209894  22.20441658  67.5884733   67.66371011
 150.97016362]


116.974832(27) = 16.743708(3) + 100.231124(+1:9)
116.974832(26) = 16.743708(2) + 100.231124(-1:9)

66.743708 (25) = 0. (1) + 66.743708(0:9)

50.693374 (23) = 16.743708 (3) + 33.949666(+1:8)
50.693374 (22) = 16.743708 (2) + 33.949666(-1:8)

66.743708 (24) = 0. (1) + 66.743708(0:8)

50. (21) = 16.743708 (3) + 33.256292(+1:7)
50. (20) = 16.743708 (2) + 33.256292(-1:7)


17.899332(17)
17.899332(16) 
  

16.974833 (15) = 0. (1) +  16.974833(0:7)
49.999999(19) = 16.743708 (3) +  33.256291(+1:6)
49.999999(18) = 16.743708(2) + 33.256291(-1:6)
16.974833 (15) = 0. (1) + 16.974833(0:6)
16.974833 (14) 
16.974833 (13)
16.974833 (12)
16.974833 (11)

16.743708 (9)
16.743708 (8)

16.974833 (10) = 16.743708 (2) + 0.231124 (-1:3)

0.693374 (3) = 0. (1) + 0.693374 (0:3)

16.743708 (7) = 16.743708 (3) + 0. (+1:2)
16.743708 (6) = 16.743708 (2) + 0. (-1:2)
0.693374 (2) = 0. (1) + 0.693374 (0:2)

16.743708 (5) = 16.743708 (3) + 0. (+1:1)
16.743708 (4) = 16.743708 (2) + 0. (-1:1)
0. (1) = 0. (1) + 0. (0:1)

In [59]:
16.743708 + 0.231124

16.974832000000003